Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal (i.e. not linear in the number of degrees of freedom). For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

We generate a 3D geometry and mesh using the OCC constructive solid geometry (CSG) modeler:

In [2]:
from netgen.occ import *
cube = Box((0,0,0),(1,1,1))
cyl = Cylinder((0,0.5,0.5),X, r=0.2, h=1)
cube.faces.name = "outer"
cyl.faces.name = "cyl"
shape = cube-cyl
Draw(shape);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

Generate a mesh, and perform uniform mesh refinement:

In [3]:
ngmesh = OCCGeometry(shape).GenerateMesh(maxh=0.1)
for l in range(1):
    ngmesh.Refine()
mesh = Mesh(ngmesh)
mesh.Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [4]:
fes = H1(mesh, order=3, dirichlet="outer", wb_withedges=False)
print ("we have", fes.ndof, "unknowns")
u,v = fes.TnT()

a = BilinearForm(grad(u)*grad(v)*dx)
f = LinearForm(v*dx)

#c = Preconditioner(a, "direct", inverse="sparsecholesky")
c = Preconditioner(a, "local")
#c = Preconditioner(a, "bddc")

gfu = GridFunction(fes)

we have 174017 unknowns


assemble system and setup preconditioner in parallel:

In [5]:
with TaskManager():
    a.Assemble()
    f.Assemble()

In [6]:
from ngsolve.la import EigenValues_Preconditioner
preI = Projector(mask=fes.FreeDofs(), range=True)

lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
print("Condition number w.o preconditioning is " + str(max(lams)/min(lams) ) )

lams = EigenValues_Preconditioner(mat=a.mat, pre=c.mat)
print("Condition number w. preconditioning is " + str(max(lams)/min(lams) ) )



Condition number w.o preconditioning is 8457.72560543067
Condition number w. preconditioning is 251.80233396973043


solve the system using the preconditioned conjugate gradient method:

In [7]:
from ngsolve.krylovspace import CGSolver

with TaskManager():
    inv = CGSolver(mat=a.mat, pre=None, freedofs = fes.FreeDofs(), printrates='\n', maxiter=10000)
    gfu.vec.data = inv * f.vec

# now with a preconditioner

with TaskManager():
    inv = CGSolver(mat=a.mat, pre=c.mat, printrates='\n', maxiter=10000)
    gfu.vec.data = inv * f.vec

CG iteration 1, residual = 0.011514129205136281     
CG iteration 2, residual = 0.026447408691575844     
CG iteration 3, residual = 0.018640092473139774     
CG iteration 4, residual = 0.015410897666923822     
CG iteration 5, residual = 0.014912002670742962     
CG iteration 6, residual = 0.014228245640239634     
CG iteration 7, residual = 0.014311594118311687     
CG iteration 8, residual = 0.01368029368845367     
CG iteration 9, residual = 0.011787285286363398     
CG iteration 10, residual = 0.009511434470068863     
CG iteration 11, residual = 0.007208013322297816     
CG iteration 12, residual = 0.005358006983387685     
CG iteration 13, residual = 0.004092532246654955     
CG iteration 14, residual = 0.003188120809397966     
CG iteration 15, residual = 0.0024912740986246604     
CG iteration 16, residual = 0.0019919575560284936     
CG iteration 17, residual = 0.0017322562654130975     
CG iteration 18, residual = 0.001619950174530207     
CG iteration 19, residual = 0.00147

In [8]:
Draw (gfu, draw_vol=False);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…